In [ ]:
from model_observers import ObserverCallback

In [ ]:
import telebot                 

token = ""                          # your bot token
temp_chat_id = ""                   # your numeric telegram id (not username!)
bot = telebot.TeleBot(token)

You can check your **user id** using function below. Simply make a new bot via [BotFather](https://telegram.me/BotFather), run this cell and send `/myid` command.

In [ ]:
@bot.message_handler(commands=['myid'])
def get_user_id(message):
    user_id = message.from_user.id
    bot.reply_to(message, f"Your user ID is: {user_id}")

bot.polling()  # Warning! Do not forget to close this cell

Lets test our callback on a MNIST dataset

In [ ]:
import tensorflow as tf
from tensorflow import keras

mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_train, y_train = X_train_full[:52000] / 255.0, y_train_full[:52000]
train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)

X_valid, y_valid = X_train_full[52000:] / 255.0, y_train_full[52000:]
valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)

X_test, y_test = X_test[:] / 255.0, y_test[:]
test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [5]:
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dense(10, activation='softmax')
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
model.fit(train_set, epochs=5, validation_data=valid_set, callbacks=[ObserverCallback(token, temp_chat_id)])

Epoch 1/5
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6663 - loss: 1.1319 - val_accuracy: 0.9511 - val_loss: 0.1726
Epoch 2/5
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9484 - loss: 0.1719 - val_accuracy: 0.9682 - val_loss: 0.1104
Epoch 3/5
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9667 - loss: 0.1099 - val_accuracy: 0.9747 - val_loss: 0.0858
Epoch 4/5
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9748 - loss: 0.0860 - val_accuracy: 0.9775 - val_loss: 0.0787
Epoch 5/5
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9789 - loss: 0.0721 - val_accuracy: 0.9786 - val_loss: 0.0728


Thats all! As you can see, all you need is to:
1. Make a **bot token**
2. Check out your **telegram id**
3. Run the model using **ObserverCallback** class